In [ ]:
import re
import os
import matplotlib.pyplot as plt
plt.style.use("default")

In [ ]:
# Experiments configuration: (Number of GPUs, Topology)
# Must match the runner script to find logs
experiments = [
    (1, "[1,1,1]"),
    (2, "[2,1,1]"),
    (3, "[3,1,1]"),
    (4, "[2,2,1]"),
]

print(f"{'GPUs':<5} | {'Topology':<10} | {'Speed (MLUPS)':<15} | {'Bandwidth (GB)':<15} | {'Bandwidth %':<12} | {'Samples':<7}")
print("-" * (5 + 3 + 10 + 3 + 15 + 3 + 15 + 3 + 12 + 3 + 7))

plot_data = []
plot_labels = []

for n_gpu, dev_dim in experiments:
    log_file = f"{n_gpu}gpu.log"

    if not os.path.exists(log_file):
        print(f"{n_gpu:<5} | {dev_dim:<10} | {'Log not found':<15} | 0")
        continue

    with open(log_file, "r") as f:
        content = f.read()

    # Extract MLUPS
    raw_speeds = [float(val) for val in re.findall(r"speed:\s*([\d\.]+)\s*\(MLUPS\)", content)]

    # Discard the first n_gpu measurements (warm-up / first step)
    assert len(raw_speeds) > n_gpu
    valid_speeds = raw_speeds[:]
    avg_speed = sum(valid_speeds) / len(valid_speeds)
    bw_ref = 4*1024**4  # Assume 4 TB/s per GPU
    bw = avg_speed * 1e6 * 27 * 2 * 4
    bw_eff_ratio = bw / bw_ref
    print(f"{n_gpu:<5} | {dev_dim:<10} | {avg_speed:<15.2f} | {bw / 1024**3:<15.2f} | {bw_eff_ratio:<12.2%} | {len(valid_speeds)}")

    plot_data.append(valid_speeds)
    plot_labels.append(n_gpu)

# Plotting  
plt.figure(figsize=(6, 4))
plt.plot(plot_labels, [max(data) for data in plot_data], marker='o', color='blue', linestyle='-', label='Ours',
         linewidth=2, markersize=8)
plt.plot([1, 2, 4], [12696, 8889, 10250], marker='o', color='red', linestyle='--', label='FluidX3D',
         linewidth=2, markersize=8)

plt.title("Performance Scaling with Number of GPUs")
plt.xlabel("Number of GPUs")
plt.ylabel("Speed (MLUPS)")
plt.ylim(2000, 15000)
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.savefig("weak_scaling_performance.png", dpi=300, bbox_inches="tight")